In [9]:
from x_xy.subpkgs import omc
import qmt
import json
import joblib
import tree
import numpy as np

_marker_closest_to_rigid_imu = {
    "seg1": 4, "seg5": 4, "seg2": 3, "seg3": 3, "seg4": 2
}

def _segment_names_in_experiment(exp_id: str) -> list[str]:
    all = set(["seg1", "seg2", "seg3", "seg4", "seg5"])
    
    if exp_id == "S_04":
        all -= set(["seg1", "seg5"])
    elif exp_id == "D_01":
        all -= set(["seg3"])
    else:
        pass

    return list(all)

def _imu_names_setup_file(exp_id: str) -> list[str]:
    if exp_id == "D_01":
        return ["imu_rigid"]
    return ["imu_rigid", "imu_flex"]

def _get_alignment(data: dict, exp_id: str, hz_omc, hz_imu):
    hz_alignment = 100.0
    hz_in = omc.hz_helper(list(data.keys()), hz_imu=hz_imu, 
                          hz_omc=hz_omc, imus=_imu_names_setup_file(exp_id))
    data = omc.resample(data, hz_in, hz_alignment)
    data = omc.crop_tail(data, hz_alignment)

    acc, gyr, mag, q, pos, names = [], [], [], [], [], []
    for seg_name, seg_data in data.items():
        imu = seg_data["imu_rigid"]
        acc.append(imu["acc"])
        gyr.append(imu["gyr"])
        mag.append(imu["mag"])
        q.append(seg_data["quat"])
        pos.append(seg_data[f"marker{_marker_closest_to_rigid_imu[seg_name]}"])
        names.append(seg_name)

    info = qmt.alignOptImu(gyr, acc, mag, q, pos, rate=hz_alignment, names=names, params=dict(fast=True))

    def _np_tolist(leaf):
        if isinstance(leaf, np.ndarray):
            return leaf.tolist()
        return leaf
    
    with open(f"alignment_infos/alignment_info_{exp_id}.json", "w") as file:
        json.dump(tree.map_structure(_np_tolist, info), file, indent=1)
    joblib.dump(info, f"alignment_infos/alignment_info_{exp_id}.joblib")

    qEOpt2EImu_euler_deg = info["qEOpt2EImu_euler_deg"]
    qImu2Seg_euler_deg = {seg_name: dict() for seg_name in data}
    for seg_name in data:
        qImu2Seg_euler_deg[seg_name]["imu_rigid"] = info[f"qImu2Seg_{seg_name}_euler_deg"]
    print(qImu2Seg_euler_deg)
    return qEOpt2EImu_euler_deg, qImu2Seg_euler_deg


def to_joblib(exp_id: str, hz_omc: int):

    folder = f"synced_aligned_nan_croptail/synced_aligned_nan_croptail_{exp_id}.joblib"
    path_optitrack_file = f"/Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/{exp_id}/optitrack/{exp_id}_{hz_omc}Hz.csv"
    path_imu_folder = f"/Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/{exp_id}/imu"
    setup_file = "/Users/simon/Documents/PYTHON/x_xy_v2/x_xy/subpkgs/exp/setups/setup.json"

    hz_omc = float(hz_omc)
    hz_imu = float(omc.utils.autodetermine_imu_freq(path_imu_folder))
    
    # perform sync
    data, imu_sync_offset = omc.read_omc(
        path_marker_imu_setup_file=setup_file,
        path_optitrack_file=path_optitrack_file,
        path_imu_folder=path_imu_folder,
        imu_names_setup_file=_imu_names_setup_file(exp_id),
        segment_names_setup_file=_segment_names_in_experiment(exp_id)
        )

    qEOpt2EImu_euler_deg, qImu2Seg_euler_deg = _get_alignment(data, exp_id, hz_omc, hz_imu)
    
    # aligned
    data, _ = omc.read_omc(
        path_marker_imu_setup_file="/Users/simon/Documents/PYTHON/x_xy_v2/x_xy/subpkgs/exp/setups/setup.json",
        path_optitrack_file=path_optitrack_file,
        path_imu_folder=path_imu_folder,
        imu_sync_offset=imu_sync_offset,
        qEOpt2EImu_euler_deg=qEOpt2EImu_euler_deg,
        qImu2Seg_euler_deg=qImu2Seg_euler_deg,
        imu_names_setup_file=_imu_names_setup_file(exp_id),
        segment_names_setup_file=_segment_names_in_experiment(exp_id)
        )
    
    hz_in = omc.hz_helper(list(data.keys()), hz_imu=hz_imu, 
                          hz_omc=hz_omc, imus=_imu_names_setup_file(exp_id))
    # resample to 100Hz, otherwise no clean crop tail *always* possible
    hz_resample = 100.0
    data = omc.resample(data, hz_in, hz_resample)
    # croptail
    data = omc.crop_tail(data, hz_resample)
    # undo resampling
    data = omc.resample(data, hz_resample, hz_in)

    joblib.dump(data, folder)

    return data

    
    

In [2]:
to_joblib("S_04", 30)

OMC Hz: 30
IMU Hz: 100
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_04/optitrack/S_04_30Hz.csv
Segment: 4, IMU: A4, Offset: 338
Segment: 4, IMU: 9B, Offset: 339
Segment: 2, IMU: A3, Offset: 338
Segment: 2, IMU: 84, Offset: 339
Segment: 3, IMU: A0, Offset: 338
Segment: 3, IMU: A2, Offset: 339
`crop_tail`: Crop off at t=647.37.
{'seg4': {'imu_rigid': array([ 0.212,  0.513, -0.083])}, 'seg2': {'imu_rigid': array([ 0.08 ,  1.017, -0.416])}, 'seg3': {'imu_rigid': array([-0.157,  1.155, -0.217])}}
OMC Hz: 30
IMU Hz: 100
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Segment: 4, IMU: A4, Offset: 338
Segment: 4, IMU: 9B, Offset: 339
Segment: 2, IMU: A3, Offset: 338
Segment: 2, IMU: 84, Offset: 339
Segment: 3, IMU: A0, Offset: 338
Segment: 3, IMU: A2, Offset: 339
`crop_tail`: Crop off at t=647.37.


{'seg4': {'imu_rigid': {'acc': array([[-0.33134017, -0.27847122,  9.81335109],
          [-0.33763106, -0.28519669,  9.85473505],
          [-0.3124099 , -0.26904838,  9.80332733],
          ...,
          [-0.32420858, -0.33042196,  9.81279723],
          [-0.3062184 , -0.33094371,  9.84210524],
          [-0.33536761, -0.31082101,  9.8191653 ]]),
   'gyr': array([[-0.00674294,  0.0038841 , -0.00337856],
          [-0.00642691,  0.00154749, -0.00735809],
          [-0.00847309,  0.00477562, -0.00480546],
          ...,
          [-0.00487549,  0.00182278, -0.0104445 ],
          [-0.00935909,  0.00289656, -0.00256165],
          [-0.0048136 ,  0.0044873 , -0.00611683]]),
   'mag': array([[ 0.10124742,  0.02929755, -0.59618889],
          [ 0.10110486,  0.01905322, -0.58909215],
          [ 0.10550086,  0.02639813, -0.58596824],
          ...,
          [ 0.09283165,  0.0367817 , -0.6332352 ],
          [ 0.09358974,  0.03092518, -0.63274529],
          [ 0.09437088,  0.02678335, -0.62

In [3]:
to_joblib("S_06", 30)

OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_06/optitrack/S_06_30Hz.csv
Segment: 5, IMU: 99, Offset: 156
Segment: 5, IMU: 85, Offset: 156
Segment: 4, IMU: A4, Offset: 156
Segment: 4, IMU: 9B, Offset: 156
Segment: 1, IMU: 9F, Offset: 156
Segment: 1, IMU: B8, Offset: 156
Segment: 2, IMU: A3, Offset: 156
Segment: 2, IMU: 84, Offset: 156
Segment: 3, IMU: A0, Offset: 156
Segment: 3, IMU: A2, Offset: 156
`crop_tail`: Crop off at t=401.0.
{'seg5': {'imu_rigid': array([ 0.271,  0.164, -1.037])}, 'seg4': {'imu_rigid': array([ 0.29 ,  0.032, -0.093])}, 'seg1': {'imu_rigid': array([0.119, 0.122, 0.309])}, 'seg2': {'imu_rigid': array([-0.268,  0.304, -0.209])}, 'seg3': {'imu_rigid': array([-0.099,  0.386, -0.22 ])}}
OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Segment: 5, IMU: 99, Offset: 156
Segment: 5, IMU: 85, Offse

{'seg5': {'imu_rigid': {'acc': array([[0.07077252, 2.39482428, 9.51896891],
          [0.09505496, 2.38017088, 9.52307878],
          [0.08780712, 2.36569718, 9.52126554],
          ...,
          [0.07296516, 2.32741637, 9.51955578],
          [0.07934591, 2.33355439, 9.53415635],
          [0.07020244, 2.33109187, 9.52930642]]),
   'gyr': array([[-0.01056986, -0.00321006, -0.01044127],
          [-0.01011197, -0.00380352, -0.01069233],
          [-0.0113738 , -0.00452506, -0.01049223],
          ...,
          [-0.01007598, -0.00333998, -0.01143365],
          [-0.00852658, -0.00453668, -0.01044992],
          [-0.01004999, -0.00556433, -0.01147705]]),
   'mag': array([[-0.83393149,  0.1957774 , -0.33600402],
          [-0.83088879,  0.20403079, -0.33408663],
          [-0.83053274,  0.20170902, -0.33294638],
          ...,
          [-0.7437605 ,  0.0982708 , -0.43423637],
          [-0.75026254,  0.0986368 , -0.43654483],
          [-0.74796516,  0.09672408, -0.43895787]])},
  'imu

In [4]:
to_joblib("S_07", 30)

OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_07/optitrack/S_07_30Hz.csv
Segment: 5, IMU: 99, Offset: 112
Segment: 5, IMU: 85, Offset: 112
Segment: 4, IMU: A4, Offset: 112
Segment: 4, IMU: 9B, Offset: 112
Segment: 1, IMU: 9F, Offset: 112
Segment: 1, IMU: B8, Offset: 112
Segment: 2, IMU: A3, Offset: 112
Segment: 2, IMU: 84, Offset: 112
Segment: 3, IMU: A0, Offset: 112
Segment: 3, IMU: A2, Offset: 112
`crop_tail`: Crop off at t=387.44.
{'seg5': {'imu_rigid': array([ 0.078,  0.326, -0.901])}, 'seg4': {'imu_rigid': array([-0.142,  0.044, -0.204])}, 'seg1': {'imu_rigid': array([-0.394,  0.102,  0.548])}, 'seg2': {'imu_rigid': array([-0.375,  0.37 , -0.133])}, 'seg3': {'imu_rigid': array([-0.414,  0.307, -0.151])}}
OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Segment: 5, IMU: 99, Offset: 112
Segment: 5, IMU: 85, O

{'seg5': {'imu_rigid': {'acc': array([[0.11849394, 2.35627467, 9.51933839],
          [0.11079431, 2.35260661, 9.51962247],
          [0.11392587, 2.35077399, 9.499219  ],
          ...,
          [0.14048425, 2.30633786, 9.52384852],
          [0.14180706, 2.29452658, 9.52543056],
          [0.1291895 , 2.29719879, 9.51131726]]),
   'gyr': array([[-0.00832435, -0.00388427, -0.01098325],
          [-0.00981201, -0.00363995, -0.01021125],
          [-0.00894484, -0.00394096, -0.01090583],
          ...,
          [-0.00870659, -0.00421725, -0.01022574],
          [-0.00968382, -0.00204191, -0.01167662],
          [-0.00851001, -0.00348604, -0.01031887]]),
   'mag': array([[-0.94823904,  0.5785633 ,  0.14179985],
          [-0.94627973,  0.57480855,  0.14196944],
          [-0.95406377,  0.58208429,  0.14885845],
          ...,
          [-0.89354624,  0.1751733 , -0.21998829],
          [-0.89749798,  0.17226108, -0.22006618],
          [-0.89851809,  0.170509  , -0.21956937]])},
  'imu

In [5]:
to_joblib("S_08", 30)

OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_08/optitrack/S_08_30Hz.csv
Segment: 5, IMU: 99, Offset: 124
Segment: 5, IMU: 85, Offset: 124
Segment: 4, IMU: A4, Offset: 124
Segment: 4, IMU: 9B, Offset: 124
Segment: 1, IMU: 9F, Offset: 124
Segment: 1, IMU: B8, Offset: 124
Segment: 2, IMU: A3, Offset: 124
Segment: 2, IMU: 84, Offset: 124
Segment: 3, IMU: A0, Offset: 124
Segment: 3, IMU: A2, Offset: 125
`crop_tail`: Crop off at t=310.3.
{'seg5': {'imu_rigid': array([ 0.238,  0.166, -1.821])}, 'seg4': {'imu_rigid': array([ 0.372,  0.556, -0.237])}, 'seg1': {'imu_rigid': array([-1.065, -0.217, -0.951])}, 'seg2': {'imu_rigid': array([-0.121,  0.374, -0.943])}, 'seg3': {'imu_rigid': array([-0.061,  0.604, -0.874])}}
OMC Hz: 30
IMU Hz: 40
IMU File Prefix: MT_0120041E-000-000_00B483
IMU File Delimiter: ;
Segment: 5, IMU: 99, Offset: 124
Segment: 5, IMU: 85, Of

{'seg5': {'imu_rigid': {'acc': array([[0.08487442, 2.51102222, 9.50274285],
          [0.07609271, 2.50054589, 9.48962087],
          [0.06992883, 2.49542852, 9.50717212],
          ...,
          [0.04131544, 2.45514057, 9.50748947],
          [0.05190683, 2.45431383, 9.50663464],
          [0.03843773, 2.47215692, 9.50611469]]),
   'gyr': array([[-0.00952498, -0.00399974, -0.01051888],
          [-0.0099574 , -0.00499506, -0.01143061],
          [-0.00894904, -0.00420664, -0.00980253],
          ...,
          [-0.00998126, -0.00397443, -0.01019706],
          [-0.00840525, -0.00253873, -0.00886658],
          [-0.00951064, -0.00346069, -0.0105942 ]]),
   'mag': array([[-0.90265926,  0.32321481, -0.23128667],
          [-0.90240878,  0.33042968, -0.23562022],
          [-0.89977804,  0.32803993, -0.24121808],
          ...,
          [-0.98024521,  0.42577866,  0.50654629],
          [-0.98567597,  0.42686439,  0.50454856],
          [-0.97982098,  0.42174222,  0.50401084]])},
  'imu

In [6]:
to_joblib("S_09", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_09/optitrack/S_09_120Hz.csv
Segment: 5, IMU: 99, Offset: 122
Segment: 5, IMU: 85, Offset: 121
Segment: 4, IMU: A4, Offset: 121
Segment: 4, IMU: 9B, Offset: 122
Segment: 1, IMU: 9F, Offset: 121
Segment: 1, IMU: B8, Offset: 122
Segment: 2, IMU: A3, Offset: 121
Segment: 2, IMU: 84, Offset: 122
Segment: 3, IMU: A0, Offset: 122
Segment: 3, IMU: A2, Offset: 121
`crop_tail`: Crop off at t=245.45.
{'seg5': {'imu_rigid': array([ 0.032,  0.248, -1.084])}, 'seg4': {'imu_rigid': array([0.555, 0.573, 0.055])}, 'seg1': {'imu_rigid': array([-0.242,  0.028,  0.769])}, 'seg2': {'imu_rigid': array([-0.042,  0.514, -0.62 ])}, 'seg3': {'imu_rigid': array([ 0.009,  0.747, -0.628])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 5, IMU: 99, Offset: 122
Segment: 5, IMU: 85, O

{'seg5': {'imu_rigid': {'acc': array([[0.09102605, 2.43307525, 9.52338722],
          [0.09900301, 2.43989959, 9.51998872],
          [0.11331629, 2.42736193, 9.50786916],
          ...,
          [0.06236131, 2.31537194, 9.54598094],
          [0.07302849, 2.30814409, 9.55292225],
          [0.06568579, 2.29948986, 9.55336758]]),
   'gyr': array([[-0.00891056, -0.00419254, -0.01173942],
          [-0.01031936, -0.00299173, -0.01019587],
          [-0.00987937, -0.00391947, -0.01096526],
          ...,
          [-0.00900246, -0.00310129, -0.01051244],
          [-0.00830106, -0.00487917, -0.0119451 ],
          [-0.00932247, -0.00358238, -0.01011568]]),
   'mag': array([[-1.02776416,  0.26465482,  0.2778018 ],
          [-1.03016055,  0.26380225,  0.27672967],
          [-1.03218712,  0.26207685,  0.27078835],
          ...,
          [-0.82250653, -0.07682478,  0.25214687],
          [-0.82408287, -0.07933647,  0.24851352],
          [-0.82285608, -0.07517198,  0.25045659]])},
  'imu

In [7]:
to_joblib("S_10", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_10/optitrack/S_10_120Hz.csv
Segment: 5, IMU: 99, Offset: 107
Segment: 5, IMU: 85, Offset: 107
Segment: 4, IMU: A4, Offset: 107
Segment: 4, IMU: 9B, Offset: 107
Segment: 1, IMU: 9F, Offset: 107
Segment: 1, IMU: B8, Offset: 107
Segment: 2, IMU: A3, Offset: 107
Segment: 2, IMU: 84, Offset: 108
Segment: 3, IMU: A0, Offset: 107
Segment: 3, IMU: A2, Offset: 107
`crop_tail`: Crop off at t=119.85.
{'seg5': {'imu_rigid': array([ 0.025,  0.212, -1.372])}, 'seg4': {'imu_rigid': array([-0.085,  0.19 ,  0.021])}, 'seg1': {'imu_rigid': array([-0.098,  0.298,  0.146])}, 'seg2': {'imu_rigid': array([-0.592,  0.323, -0.536])}, 'seg3': {'imu_rigid': array([-0.138,  0.404, -0.687])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 5, IMU: 99, Offset: 107
Segment: 5, IMU: 85

{'seg5': {'imu_rigid': {'acc': array([[0.10037022, 2.30783424, 9.53260627],
          [0.0961845 , 2.29841843, 9.54963337],
          [0.11107991, 2.31027997, 9.55170981],
          ...,
          [0.05201588, 2.36382493, 9.53169595],
          [0.09678223, 2.34834758, 9.53279846],
          [0.08633362, 2.35667393, 9.52836512]]),
   'gyr': array([[-0.01067952, -0.00498852, -0.01016914],
          [-0.00990172, -0.00365488, -0.01104   ],
          [-0.0083304 , -0.00286236, -0.01069887],
          ...,
          [-0.00852231, -0.00329077, -0.00777905],
          [-0.00879336, -0.00327229, -0.01398031],
          [-0.00934534, -0.00438588, -0.01015799]]),
   'mag': array([[-0.80679766, -0.03557762,  0.020924  ],
          [-0.80266038, -0.039395  ,  0.02354011],
          [-0.80850677, -0.03533344,  0.02092445],
          ...,
          [-0.80835753, -0.03240541,  0.60918556],
          [-0.8068029 , -0.03310497,  0.61356777],
          [-0.80929302, -0.03983771,  0.61290768]])},
  'imu

In [8]:
to_joblib("S_12", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_12/optitrack/S_12_120Hz.csv
Segment: 5, IMU: 99, Offset: 92
Segment: 5, IMU: 85, Offset: 91
Segment: 4, IMU: A4, Offset: 90
Segment: 4, IMU: 9B, Offset: 92
Segment: 1, IMU: 9F, Offset: 90
Segment: 1, IMU: B8, Offset: 92
Segment: 2, IMU: A3, Offset: 90
Segment: 2, IMU: 84, Offset: 91
Segment: 3, IMU: A0, Offset: 91
Segment: 3, IMU: A2, Offset: 91
`crop_tail`: Crop off at t=330.29.
{'seg5': {'imu_rigid': array([ 0.057,  0.383, -0.993])}, 'seg4': {'imu_rigid': array([-0.154,  0.366,  0.205])}, 'seg1': {'imu_rigid': array([ 0.051, -0.063,  0.078])}, 'seg2': {'imu_rigid': array([-0.152,  0.467, -0.921])}, 'seg3': {'imu_rigid': array([ 0.031,  0.535, -0.012])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 5, IMU: 99, Offset: 92
Segment: 5, IMU: 85, Offset: 9

{'seg5': {'imu_rigid': {'acc': array([[0.09885992, 7.91218661, 5.79691432],
          [0.11156808, 7.89548167, 5.79195556],
          [0.11824769, 7.89054828, 5.79838047],
          ...,
          [0.13663899, 7.4869738 , 6.30991634],
          [0.13473433, 7.51055812, 6.30645306],
          [0.13148991, 7.50295495, 6.28251608]]),
   'gyr': array([[-0.01226133, -0.00301405, -0.01107792],
          [-0.00985557, -0.00387001, -0.00954335],
          [-0.00975684, -0.00250494, -0.01123647],
          ...,
          [-0.01001308, -0.00250995, -0.01008958],
          [-0.0095174 , -0.00339881, -0.01100553],
          [-0.00991149, -0.00247794, -0.01017642]]),
   'mag': array([[0.42872134, 0.36576109, 0.49283887],
          [0.43888007, 0.36644886, 0.48956026],
          [0.4407253 , 0.37126693, 0.49974459],
          ...,
          [0.20767856, 0.52370122, 0.51013191],
          [0.21484411, 0.52463544, 0.5156842 ],
          [0.21220263, 0.5202668 , 0.52161357]])},
  'imu_flex': {'acc': ar

In [9]:
to_joblib("S_13", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_13/optitrack/S_13_120Hz.csv
Segment: 5, IMU: 99, Offset: 102
Segment: 5, IMU: 85, Offset: 102
Segment: 4, IMU: A4, Offset: 102
Segment: 4, IMU: 9B, Offset: 102
Segment: 1, IMU: 9F, Offset: 102
Segment: 1, IMU: B8, Offset: 102
Segment: 2, IMU: A3, Offset: 102
Segment: 2, IMU: 84, Offset: 102
Segment: 3, IMU: A0, Offset: 102
Segment: 3, IMU: A2, Offset: 102
`crop_tail`: Crop off at t=396.52.
{'seg5': {'imu_rigid': array([ 0.175,  0.248, -1.063])}, 'seg4': {'imu_rigid': array([-0.329,  0.348,  0.13 ])}, 'seg1': {'imu_rigid': array([-0.201,  0.004, -0.084])}, 'seg2': {'imu_rigid': array([-0.195,  0.178, -0.865])}, 'seg3': {'imu_rigid': array([-0.212,  0.627,  0.151])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 5, IMU: 99, Offset: 102
Segment: 5, IMU: 85

{'seg5': {'imu_rigid': {'acc': array([[0.08899027, 7.51679796, 6.29759399],
          [0.09077826, 7.51053947, 6.26685386],
          [0.08097059, 7.52925262, 6.31595335],
          ...,
          [0.02371312, 7.46409103, 6.35969055],
          [0.03799922, 7.47233957, 6.33127255],
          [0.00916323, 7.46046596, 6.35096741]]),
   'gyr': array([[-0.01058946, -0.00373655, -0.01033328],
          [-0.0105146 , -0.00218587, -0.01130516],
          [-0.00915827, -0.00153814, -0.01035821],
          ...,
          [-0.01075745, -0.0030903 , -0.00895031],
          [-0.00891696, -0.00318791, -0.01092101],
          [-0.00994085, -0.00305474, -0.01040572]]),
   'mag': array([[0.20426288, 0.52304797, 0.51856436],
          [0.20604422, 0.52071737, 0.51107866],
          [0.20671666, 0.52063298, 0.51957489],
          ...,
          [0.48034261, 0.27534081, 0.55836811],
          [0.47635654, 0.26727361, 0.56251456],
          [0.47713168, 0.2719537 , 0.54745573]])},
  'imu_flex': {'acc': ar

In [2]:
to_joblib("S_14", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_14/optitrack/S_14_120Hz.csv
Segment: 4, IMU: A4, Offset: 96
Segment: 4, IMU: 9B, Offset: 96
Segment: 1, IMU: 9F, Offset: 96
Segment: 1, IMU: B8, Offset: 96
Segment: 5, IMU: 99, Offset: 96
Segment: 5, IMU: 85, Offset: 96
Segment: 3, IMU: A0, Offset: 96
Segment: 3, IMU: A2, Offset: 95
Segment: 2, IMU: A3, Offset: 96
Segment: 2, IMU: 84, Offset: 96
`crop_tail`: Crop off at t=248.59000000000006.


/Users/simon/Documents/PYTHON/x_xy_v2/x_xy/subpkgs/omc/utils.py:49: UserWarning: Must crop more than i=101 and still no clean crop possible.
  warnings.warn(


{'seg4': {'imu_rigid': array([-0.103,  0.606,  0.22 ])}, 'seg1': {'imu_rigid': array([-0.068,  0.084,  0.228])}, 'seg5': {'imu_rigid': array([-0.091,  0.287, -1.101])}, 'seg3': {'imu_rigid': array([-0.013,  0.861, -0.159])}, 'seg2': {'imu_rigid': array([-0.014,  0.348, -0.828])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: ;
Segment: 4, IMU: A4, Offset: 96
Segment: 4, IMU: 9B, Offset: 96
Segment: 1, IMU: 9F, Offset: 96
Segment: 1, IMU: B8, Offset: 96
Segment: 5, IMU: 99, Offset: 96
Segment: 5, IMU: 85, Offset: 96
Segment: 3, IMU: A0, Offset: 96
Segment: 3, IMU: A2, Offset: 95
Segment: 2, IMU: A3, Offset: 96
Segment: 2, IMU: 84, Offset: 96
`crop_tail`: Crop off at t=248.59000000000006.


/Users/simon/Documents/PYTHON/x_xy_v2/x_xy/subpkgs/omc/utils.py:49: UserWarning: Must crop more than i=101 and still no clean crop possible.
  warnings.warn(


{'seg4': {'imu_rigid': {'acc': array([[ 0.31179246, -6.0929796 ,  7.7372662 ],
          [ 0.31450359, -6.10701069,  7.71777887],
          [ 0.30712811, -6.0744726 ,  7.73042548],
          ...,
          [-9.05075623, -2.4025596 , -5.90191225],
          [-9.02997102, -2.2291    , -5.8686117 ],
          [-9.00426784, -2.92637965, -6.51494819]]),
   'gyr': array([[-9.90313701e-03,  5.25094064e-04, -4.39758098e-03],
          [-5.11365722e-03,  3.22472273e-03, -2.99339738e-03],
          [-4.19559129e-03,  2.89773663e-03, -3.64570392e-03],
          ...,
          [ 9.38828732e-02,  1.10605283e+00, -1.20300360e-01],
          [ 9.85871651e-02,  7.81773806e-01, -1.64592789e-01],
          [-1.20706639e-01,  4.67531754e-01, -2.04408056e-01]]),
   'mag': array([[-0.31609538,  0.40742223, -0.40746669],
          [-0.3137426 ,  0.40797011, -0.41134689],
          [-0.31762425,  0.41477806, -0.41474661],
          ...,
          [ 0.79895693, -0.01988834,  0.47978995],
          [ 0.7921283

In [3]:
to_joblib("S_15", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_15/optitrack/S_15_120Hz.csv
Segment: 4, IMU: A4, Offset: 118
Segment: 4, IMU: 9B, Offset: 118
Segment: 1, IMU: 9F, Offset: 118
Segment: 1, IMU: B8, Offset: 118
Segment: 5, IMU: 99, Offset: 118
Segment: 5, IMU: 85, Offset: 118
Segment: 3, IMU: A0, Offset: 118
Segment: 3, IMU: A2, Offset: 118
Segment: 2, IMU: A3, Offset: 118
Segment: 2, IMU: 84, Offset: 118
`crop_tail`: Crop off at t=246.18.
{'seg4': {'imu_rigid': array([-0.144,  0.401,  0.177])}, 'seg1': {'imu_rigid': array([-0.234,  0.358,  0.423])}, 'seg5': {'imu_rigid': array([-0.22 ,  0.259, -0.743])}, 'seg3': {'imu_rigid': array([-0.114,  1.06 , -0.037])}, 'seg2': {'imu_rigid': array([-0.181,  0.615, -0.487])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Segment: 4, IMU: A4, Offset: 118
Segment: 4, IMU: 9B

{'seg4': {'imu_rigid': {'acc': array([[ 0.51502809, -5.53520642,  8.09791464],
          [ 0.53904034, -5.54569817,  8.10652463],
          [ 0.52768996, -5.54063379,  8.12680011],
          ...,
          [ 5.13131214, -5.95097834,  5.93129266],
          [ 5.12653849, -5.9552159 ,  5.91786638],
          [ 5.13620015, -5.94998598,  5.9424782 ]]),
   'gyr': array([[-0.00506821,  0.00326746, -0.00989971],
          [-0.00481028,  0.00106063, -0.00897544],
          [-0.00805058,  0.00159227, -0.00933905],
          ...,
          [-0.00641484,  0.00166051, -0.00897776],
          [-0.00479995,  0.00437961, -0.00854809],
          [-0.00528671, -0.00037509, -0.00780503]]),
   'mag': array([[-0.30458661,  0.56767852, -0.39748563],
          [-0.30575732,  0.56177686, -0.39869192],
          [-0.30245614,  0.56330306, -0.40557136],
          ...,
          [-0.42533229,  0.52907089, -0.23232842],
          [-0.42465457,  0.52817783, -0.23636425],
          [-0.42348578,  0.53208672, -0.23

In [4]:
to_joblib("S_16", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/S_16/optitrack/S_16_120Hz.csv
Segment: 4, IMU: A4, Offset: 155
Segment: 4, IMU: 9B, Offset: 156
Segment: 1, IMU: 9F, Offset: 155
Segment: 1, IMU: B8, Offset: 155
Segment: 5, IMU: 99, Offset: 155
Segment: 5, IMU: 85, Offset: 156
Segment: 3, IMU: A0, Offset: 155
Segment: 3, IMU: A2, Offset: 156
Segment: 2, IMU: A3, Offset: 155
Segment: 2, IMU: 84, Offset: 155
`crop_tail`: Crop off at t=185.0.
{'seg4': {'imu_rigid': array([-0.529,  0.002, -0.194])}, 'seg1': {'imu_rigid': array([-0.68 , -0.67 ,  0.114])}, 'seg5': {'imu_rigid': array([ 0.203,  0.427, -0.724])}, 'seg3': {'imu_rigid': array([ 0.047,  1.225, -0.564])}, 'seg2': {'imu_rigid': array([-0.112,  0.66 , -0.473])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Segment: 4, IMU: A4, Offset: 155
Segment: 4, IMU: 9B,

{'seg4': {'imu_rigid': {'acc': array([[-4.80783340e-01,  1.14714458e-01,  9.82715579e+00],
          [-4.69615904e-01,  1.14773259e-01,  9.82765845e+00],
          [-4.70478247e-01,  1.11389901e-01,  9.80847425e+00],
          ...,
          [-2.95974245e-03,  8.74025624e-02,  9.84414563e+00],
          [ 6.51076736e-03,  6.21435590e-02,  9.81876247e+00],
          [-2.29308426e-03,  6.81414774e-02,  9.83442934e+00]]),
   'gyr': array([[-0.00526602,  0.00254603, -0.00877532],
          [-0.00690532,  0.00229467, -0.00900556],
          [-0.0062154 ,  0.00034042, -0.00796779],
          ...,
          [-0.00598477,  0.00177464, -0.00865276],
          [-0.00763133,  0.00154329, -0.00715976],
          [-0.00643383,  0.00272494, -0.00903395]]),
   'mag': array([[ 0.2705033 , -0.61380164,  1.02525898],
          [ 0.26875183, -0.61576479,  1.03195568],
          [ 0.26484825, -0.61812971,  1.02966904],
          ...,
          [ 0.26705246, -0.94295439,  0.41411434],
          [ 0.2665537

In [10]:
to_joblib("D_01", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Segment: 4, IMU: A4, Offset: 133
Segment: 1, IMU: 9F, Offset: 133
Segment: 5, IMU: 99, Offset: 133
Segment: 2, IMU: A3, Offset: 133
`crop_tail`: Crop off at t=474.92.
{'seg4': {'imu_rigid': array([-0.132,  0.32 ,  0.004])}, 'seg1': {'imu_rigid': array([0.055, 0.33 , 0.161])}, 'seg5': {'imu_rigid': array([ 0.52 ,  0.459, -0.939])}, 'seg2': {'imu_rigid': array([-0.129,  0.738, -0.639])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Segment: 4, IMU: A4, Offset: 133
Segment: 1, IMU: 9F, Offset: 133
Segment: 5, IMU: 99, Offset: 133
Segment: 2, IMU: A3, Offset: 133
`crop_tail`: Crop off at t=474.92.


{'seg4': {'imu_rigid': {'acc': array([[0.05391619, 0.18806275, 9.81946461],
          [0.03653714, 0.20509222, 9.83551362],
          [0.0408144 , 0.20491849, 9.81223406],
          ...,
          [0.07401207, 0.25097395, 9.82887772],
          [0.06322173, 0.25025807, 9.83802817],
          [0.01883397, 0.27096459, 9.82258444]]),
   'gyr': array([[-8.62307866e-03,  2.73646708e-03, -8.45574706e-03],
          [-4.63664774e-03,  3.18035344e-03, -8.03756745e-03],
          [-6.46792372e-03,  2.86244114e-03, -7.59175938e-03],
          ...,
          [-8.03050076e-03,  6.22978079e-04, -6.76620490e-03],
          [ 3.75801310e-05,  3.66465018e-03, -7.54932465e-03],
          [ 1.49663277e-03,  2.99517556e-03, -8.82624851e-03]]),
   'mag': array([[-0.66315388, -0.24799105,  0.09230755],
          [-0.66661717, -0.24839741,  0.08434303],
          [-0.66609844, -0.25018115,  0.09061479],
          ...,
          [-0.57704583, -0.05286023, -0.13276973],
          [-0.58135133, -0.05338702, -0

In [11]:
to_joblib("T_01", 120)

OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Loading OMC data from file /Users/simon/Documents/berlin_02_06_23/berlin_02_06_23/experiments/T_01/optitrack/T_01_120Hz.csv
Segment: 4, IMU: A4, Offset: 70
Segment: 4, IMU: 9B, Offset: 70
Segment: 1, IMU: 9F, Offset: 70
Segment: 1, IMU: B8, Offset: 70
Segment: 5, IMU: 99, Offset: 70
Segment: 5, IMU: 85, Offset: 70
Segment: 3, IMU: A0, Offset: 70
Segment: 3, IMU: A2, Offset: 70
Segment: 2, IMU: A3, Offset: 70
Segment: 2, IMU: 84, Offset: 70
`crop_tail`: Crop off at t=216.09.
{'seg4': {'imu_rigid': array([-0.098,  0.77 ,  0.06 ])}, 'seg1': {'imu_rigid': array([-0.038,  0.446, -0.233])}, 'seg5': {'imu_rigid': array([ 0.233,  0.639, -1.199])}, 'seg3': {'imu_rigid': array([ 0.192,  0.612, -0.193])}, 'seg2': {'imu_rigid': array([ 0.166,  0.643, -0.866])}}
OMC Hz: 120
IMU Hz: 40
IMU File Prefix: MT_012102D5-000-000_00B483
IMU File Delimiter: 	
Segment: 4, IMU: A4, Offset: 70
Segment: 4, IMU: 9B, Offset: 7

{'seg4': {'imu_rigid': {'acc': array([[ 0.23551299, -5.9963551 ,  7.80702654],
          [ 0.2197111 , -5.99102991,  7.81215023],
          [ 0.22962851, -5.99985072,  7.8172988 ],
          ...,
          [ 0.21591499, -5.60063487,  8.087004  ],
          [ 0.21118791, -5.59654365,  8.10433127],
          [ 0.2225079 , -5.60636408,  8.08454876]]),
   'gyr': array([[-0.00609137,  0.00207613, -0.00441733],
          [-0.00548596,  0.00205275, -0.00542978],
          [-0.00714178,  0.00127649, -0.00493412],
          ...,
          [-0.00674381,  0.00234725, -0.00488952],
          [-0.00590641,  0.00120387, -0.00542684],
          [-0.00658128,  0.00243314, -0.00419185]]),
   'mag': array([[-0.46523421,  0.27483066, -0.09722345],
          [-0.45945873,  0.27830531, -0.09121759],
          [-0.46685809,  0.27653604, -0.09109525],
          ...,
          [-0.5128204 ,  0.05605756, -0.34723158],
          [-0.51457578,  0.0505623 , -0.34267176],
          [-0.50755358,  0.05543131, -0.34